In [65]:
import sys
import time
import pandas as pd
from langchain_ollama import OllamaEmbeddings
from ollama import Client
import json
import csv
import ast

In [66]:
CONFLICT_PROMPT_ODL = """You are tasked with determining if two OpenDaylight (ODL) flow configuration JSONs directly conflict with each other. You MUST base your decision SOLELY on the content of the JSON structures provided. Do NOT consider the intent behind the flow rule or try to infer conflicts based on the general idea of blocking or forwarding traffic. Your analysis MUST be a strict, field-by-field comparison of the JSON structures.

### **Rules of Conflict Detection (STRICT and LITERAL):


A **direct conflict** exists ONLY if *all* of the following conditions are met:

1. **Exact Matching Traffic Characteristics (Match Criteria Overlap):** Both flows MUST have *identical* or *overlapping* match criteria for *all specified fields*. This means they could potentially apply to the *exact same* packet. Critically, if a field is present in one flow but *not* in the other, there is *NO MATCH* for that field. Consider the following match fields:
    *   `in-port`: Both flows must have the `in-port` field specified and the values must be identical for an overlap. If either flow does not have the `in-port` field, there is NO overlap for this criterion.
    *   `ethernet-match.ethernet-type.type`: Both flows must have this field and the values must be identical (e.g., 2048 for IPv4) for an overlap. If either flow does not have this field, there is NO overlap for this criterion.
    *   `ipv4-source`: Both flows must have the `ipv4-source` field specified. The source IP addresses/masks must overlap (e.g., 10.0.0.1/32 and 10.0.0.0/24 overlap) for an overlap. If either flow does not have the `ipv4-source` field, there is NO overlap for this criterion.
    *   `ipv4-destination`: Both flows must have the `ipv4-destination` field specified. The destination IP addresses/masks must overlap for an overlap. If either flow does not have the `ipv4-destination` field, there is NO overlap for this criterion.
    *   `ip-match.ip-protocol`: Both flows must have the `ip-protocol` field specified and the values must be identical (e.g., 6 for TCP, 17 for UDP) for an overlap. If either flow does not have this field, there is NO overlap for this criterion.
    *   `tcp-source-port`, `tcp-destination-port`, `udp-source-port`, `udp-destination-port`: Both flows must have the *same* port field (e.g., both `tcp-destination-port`) specified, and the values must be identical for an overlap. If either flow does not have the corresponding port field, there is NO overlap for this criterion.
    *    **EXCEPTION** If one flow has additional match fields for ethernet-type or ip-protocol or tcp-destination-port or udp-destination-port, then there is NO overlap for these flows even if the output actions are same and some but not all required fields (such as ethernet-type, ip-protocol, IP address, tcp-destination-port, udp-destination-port) match.

2. **Contradictory Actions:** If *and only if* the match criteria overlap as described above, check the actions. Contradictory actions include:
    *   Different `output-node-connector` values: If the flows have different output ports, it's a conflict..
    *   One flow has `drop-action: {}` and the other has an `output-action`: If one flow drops the packet and the other forwards it, it's a conflict.
    *   Different `set-queue-action.queue-id` values: If the flows specify different queue IDs, it's a conflict.
    *   **EXCEPTION** If one flow has additional match fields for ethernet-type or ip-protocol or tcp-destination-port or udp-destination-port, then there is NO overlap for these flows even if the output actions are same and some but not all required fields (such as ethernet-type, ip-protocol, IP address, tcp-destination-port, udp-destination-port) match.

3. **Priority is Irrelevant for Direct Conflicts:** The `priority` field does *not* determine if a direct conflict exists. Priority only determines which flow rule takes precedence if a conflict is detected by the switch.

4. **No General vs. Specific Rule Heuristics**  
    Do NOT assume that one rule is “more general” or “more specific.” Only literal match-set overlap matters. Two rules conflict if — and only if — they match the same packet and specify different actions.
    
5. **Source to Source, Destination to Destination:** You MUST match source to source and destination to destination. Any match between a JSON's source to another JSON's destination is NOT a conflict.

6. **Missing Field Handling - EXTREMELY IMPORTANT (LITERAL COMPARISON):**
    *   If one JSON omits a field (e.g., ipv4-source), it matches all values for that field. Therefore, if the other JSON specifies a narrower match, and their actions differ, this can be a conflict.
    *   **Crucially:** You MUST perform a literal, field-by-field comparison. Do NOT infer any broader meaning or intent.
    *   **Example 1 (CONFLICT):**
        *   Flow 1: `{"match": {"ipv4-destination": "10.0.0.2/32"}, "instructions": { ... "output-action": ... }}`
        *   Flow 2: `{"match": {"ipv4-source": "10.0.0.1/32", "ipv4-destination": "10.0.0.2/32"}, "instructions": { ... "drop-action": {} }}`
        *   **Explanation:** Flow 1 matches all sources to 10.0.0.2. Flow 2 matches only packets from 10.0.0.1 to 10.0.0.2. Their match sets may overlap — so if their actions differ, this is a conflict. 
   *     **Example 2 (NO CONFLICT):**
         *  One flow allows traffic to `10.0.0.2/32` 
         *  Another flow blocks traffic from `10.0.0.2/32` to `10.0.0.4/32`
         *   **Explanation:** These do not conflict because source and destination fields do not align.
   *   **Example 3 (NO CONFLICT):**
        *   Flow 1: `{"match": {"ipv4-source": "10.0.0.1/32", "ipv4-destination": "10.0.0.4/32"}, "instructions": { ... "drop-action": {} }}`
        *   Flow 2: `{"match": {"ipv4-source": "10.0.0.2/32", "ipv4-destination": "10.0.0.4/32"}, "instructions": { ... "drop-action": {} }}`
        *   **Explanation:** Although both flows drop traffic to 10.0.0.4, they have *different sources*. Therefore, they do NOT conflict. They apply to different traffic.
   *     **Example 4 (NO CONFLICT):**
         *  One flow has destination IP `10.0.0.1` 
         *  Another flow has detination IP `10.0.0.2`
         *   **Explanation:** These do not conflict even if the output actions are different because the source and the destination IP are not same, i.e. 10.0.0.1/32 and 10.0.0.2/32 do not overlap as 10.0.0.1 is not within the subnet of 10.0.0.2/32.

### **Input Format:**

You will be provided with two JSON objects representing ODL flow entries. The input will be formatted as follows:

Flow 1:
<JSON for Flow 1>

Flow 2:
<JSON for Flow 2>

### **Expected Output Format:**

Respond strictly in valid JSON adhering to the following schema:

```json
{
  "conflict_status": <integer>,
  "conflict_explanation": "<conflict explanation, if any>"
}

Field Descriptions:

conflict_status: An integer value: 1 if a direct conflict exists, 0 if no direct conflict exists. **DO NOT PUT 1 IF YOU ARE NOT CONFIDENT ABOUT A DIRECT CONFLICT**.
conflict_explanation: A brief explanation of the detected conflict, if any. This field should be an empty string ("") if there is no conflict.
Do not include any additional text, comments, or explanations outside of the JSON structure. Only output valid JSON."""


In [ ]:
# model names should match the downloaded ollama model names. 
# If ollama does not find a model, check ollama model list (write in terminal: "ollama list" without quote) and model names. Use the correct model name.

my_models=[
    "codegemma:7b",
    "codestral:22b",
    "codellama:34b",
    "codellama:7b",
    "command-r:35b",
    "deepseek-coder:1.3b",
    "Deepseek-coder-v2:16b",
    "dolphin-mistral:7b",
    "gemma2:27b",
    "huihui_ai/qwq-abliterated:latest",
    "huihui_ai/qwq-fusion:32b",
    "llama2:7b",
    "llama3:latest",
    "llama3.1:8b",
    "llama3.2:3b",
    "llava-llama3:8b",
    "marco-o1:7b",
    "mistral:latest",
    "mistral-nemo:12b",
    "openchat:7b",
    "orca-mini:3b",
    "phi:2.7b",
    "phi3:3.8b",
    "qwen:4b",
    "qwen2:7b",
    "qwen2.5:7b",
    "qwq:latest",
    "starcoder:3b",
    "starcoder2:3b",
    "TinyLlama:1.1b",
    "wizardlm2:7b",
    "yi:6b",
    "zephyr:7b"
]


default_model = "llama2:7b"

#replace with your host IP or use localhost. Make sure the correct port number of ollama server.
ollama_embedding_url = "http://10.23.7.63:11434"
ollama_server_url = "http://10.23.7.63:11435"  

ollama_emb = OllamaEmbeddings(
    model=default_model,
    base_url=ollama_embedding_url,
)

client = Client(host=ollama_server_url , timeout=120)

In [68]:
def run_LLM_conflict(flow_json_1, flow_json_2, model):
    
    system_prompt = CONFLICT_PROMPT_ODL
    count = 0
    while True:
        count+=1
        try:
            time.sleep(0.1)

            response = client.generate(model=model,
                options={'temperature': 0.3, 'num_ctx': 8192, 'top_p': 0.5, 'num_predict': 1024, 'num_gpu': 99},
                stream=False,
                system=system_prompt,
                prompt=f"Flow 1:\n{json.dumps(flow_json_1, indent=2)}\n\nFlow 2:\n{json.dumps(flow_json_2, indent=2)}",
                format='json'
            )

            output = response['response'].strip()

            response_json = json.loads(output)

            if 'conflict_status' not in response_json:
                #print("\nWarning: 'conflict_status' key is missing in the response.\n")
                break
            else:
                valid_conflict_response = True
                conflict_status = response_json.get('conflict_status', 0)
                # Ensure conflict_status is an integer
                if isinstance(conflict_status, str):
                    conflict_status = int(conflict_status)

                return valid_conflict_response, conflict_status, response_json['conflict_explanation']             

        except Exception as e:
            print("Exception found: ", e)
            sys.stdout.flush()
            if(count<15):
                continue
            else:
                print("\n",model, " failed to produce valid JSON for conflict info after 15 tries. Going to next model\n")
                break               
    
    return False, None, None

In [69]:
def safe_load_flow_json(flow_str, index_number):
    # Try ast.literal_eval first
    try:
        parsed = ast.literal_eval(flow_str)
        return parsed
    except Exception as e1:
        # Try JSON parsing with cleanup for double-double quotes
        try:
            cleaned_str = flow_str.replace('""', '"')
            parsed = json.loads(cleaned_str)
            return parsed
        except Exception as e2:
            print(f"[Index {index_number}] Failed to parse flow string.\n"
                  f"ast.literal_eval error: {e1}\n"
                  f"json.loads error: {e2}")
            return None

In [ ]:
df = pd.read_excel("FlowConflict-ODL.xlsx")

summary_rows = []

for model in my_models:
    print("Now Model: ", model, "\n\n")
    TP = FP = TN = FN = 0
    times = []

    for idx, row in df.iterrows():

        index_number = row["Sl."]
        flow_rule_1_json = safe_load_flow_json(row["ODL Flow Rule 1"], index_number)
        flow_rule_2_json = safe_load_flow_json(row["ODL Flow Rule 2"], index_number)

        ground_truth_conflict = row["Conflicting"].strip().lower()  # 'yes' or 'no'
        gt_conflict = (ground_truth_conflict == 'yes')

        if (flow_rule_1_json is None) or (flow_rule_2_json is None):
            continue

        start_conflict_time = time.time()
        valid_conflict_response, conflict_status, conflict_details = run_LLM_conflict(
            flow_rule_1_json, flow_rule_2_json, model=model)
        conflict_duration = time.time() - start_conflict_time
        times.append(conflict_duration)

        # Only score if a valid result was returned
        if conflict_status == 2 or not valid_conflict_response:
            continue
        detected_conflict = (conflict_status == 1)

        # Update counts
        if detected_conflict and gt_conflict:
            TP += 1
        elif detected_conflict and not gt_conflict:
            FP += 1
        elif not detected_conflict and not gt_conflict:
            TN += 1
        elif not detected_conflict and gt_conflict:
            FN += 1

    total = TP + TN + FP + FN
    accuracy = (TP + TN) / total if total > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    fpr = FP / (FP + TN) if (FP + TN) > 0 else 0
    avg_time = sum(times) / len(times) if times else 0

    summary_rows.append([
        model, TP, FP, TN, FN, round(accuracy, 4), round(precision, 4),
        round(recall, 4), round(f1, 4), round(fpr, 4), round(avg_time, 4)
    ])

# ---- Write summary CSV ----
with open("ODL_conflict_detection_model_summaries.csv", 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Model", "TP", "FP", "TN", "FN", "Accuracy", "Precision", "Recall", "F1", "FPR", "Avg Time (s)"])
    for row in summary_rows:
        writer.writerow(row)

print("Processing completed. Model summaries saved to CSV.")

Now Model:  codegemma:7b 


Now Model:  codestral:22b 


Now Model:  codellama:34b 


